In [7]:
# 行专列：https://www.cnblogs.com/traditional/p/11967360.html
# 计算行列总和：https://blog.csdn.net/weixin_43745169/article/details/103372098

import pandas as pd
#from Texttable import texttable

#import xlwt as xw

START_DATE=221031
END_DATE=221113
DESIGNER=str(START_DATE)+'-'+str(END_DATE)

data=pd.read_excel("video-data2.xlsx")
data=data[data['制作时间']>=START_DATE]
data=data[data['制作时间']<=END_DATE]

data=data.rename(columns={'设计师':DESIGNER})

writer=pd.ExcelWriter('output.xlsx')

#get data_class(in_data,column_name):
column_name='素材类型'
in_data=data
in_data=in_data.groupby([DESIGNER,column_name])
name_data=in_data['素材名'].apply(lambda x: "、".join(list(set(x.str.cat(sep=',').split(','))))).unstack()
count_data=in_data['素材名'].count().unstack()
count_data['总计']=count_data.apply(lambda x: x.sum(),axis=1)
count_data.loc['总计']=count_data.apply(lambda x: x.sum(),axis=0)

name_data.to_excel(writer,'name')
count_data.to_excel(writer,'count')
    

#data_class(data,'素材类型')


writer.save()


print('done')

done


In [6]:
# 参考
# https://blog.csdn.net/baidu_41797613/article/details/121555351
# 排序：https://my.oschina.net/u/4368331/blog/3459650

import pandas as pd
import numpy as np


# 1、process video file, get the name of each category in period (output to different files)


video=pd.read_excel("video-data2.xlsx")
market=pd.read_excel("market-221116-2.xlsx")
market.sort_values(by=['消耗'],axis=0, ascending=False,inplace=True)

# START_DATE=221024
# END_DATE=221106
START_DATE=221031
END_DATE=221113
CATEGORY='素材类型'
OUTPUT_NAME=CATEGORY+"-VIDEO-ANALYSIS.xlsx"
# '用户心理需求'
# '开头'
# '素材类型'
# '主卖点'
# '副卖点'
# '创意形式'
# '创意架构'
# '游戏付费点'


    


def get_pt(pt):
    new_pt = pd.DataFrame()
    new_pt['消耗'] = round(pt['消耗'],0)

    new_pt['激活成本'] = round(pt['消耗'] / pt['激活数'], 0)
    new_pt['付费成本'] = round(pt['消耗'] / pt['首日付费人数'], 0)
    new_pt['点击率%'] = round(pt['点击数'] / pt['曝光数'], 4) * 100
    new_pt['转化率%'] = round(pt['激活数'] / pt['点击数'], 3) * 100
    new_pt['注册率%'] = round(pt['注册数'] / pt['激活数'], 3) * 100
    new_pt['首日付费率%'] = round(pt['首日付费人数'] / pt['激活数'], 3) * 100
    new_pt['次留率%'] = round(pt['次留数'] / pt['激活数'], 3) * 100
    new_pt['首日ROI%'] = round(pt['首日付费金额'] / pt['消耗'], 3) * 100
    new_pt['三日ROI%'] = round(pt['F3付费金额'] / pt['消耗'], 3) * 100
    new_pt['3/1涨幅'] = round(new_pt['三日ROI%'] / new_pt['首日ROI%'], 2)
    
    new_pt['25%进度播放率%']=round(pt['25%进度播放数'] /pt['曝光数'], 3) * 100
    new_pt['50%进度播放率%']=round(pt['50%进度播放数'] /pt['曝光数'], 3) * 100
    new_pt['75%进度播放率%']=round(pt['75%进度播放数'] /pt['曝光数'], 3) * 100
    new_pt['完播率%']=round(pt['99%进度播放数'] /pt['曝光数'], 3) * 100
    new_pt['有效播放率%']=round(pt['有效播放数'] /pt['曝光数'], 3) * 100

    return new_pt

def style_excel(data,pt_header,sn,sort_index,sort_value):
    pt=pd.pivot_table(data,index=pt_header,aggfunc=np.sum,margins=False)
    pt_sorted=get_pt(pt)
    pt_sorted[np.isinf(pt_sorted)]=0
    pt_sorted.sort_values(by=sort_index,ascending=sort_value,inplace=True)
    
    
    pt_style=pt_sorted.fillna(0).style.background_gradient(cmap='Blues')\
            .background_gradient(cmap='Reds',subset=['首日ROI%'])\
            .background_gradient(cmap='Reds',subset=['三日ROI%'])\


    pt_style.to_excel(writer,sheet_name=sn,index=True)
    print(sn,"done!")
    
def get_type(market_d,video_d,CATEGORY_d):
    type_pd_d=pd.DataFrame()
    
    video_category_list=video_d[CATEGORY_d].unique()
    
    
    for TYPE in video_category_list:
        #print(f'--------------------------{TYPE}-------------------------------------')

        video_type = video_d[video_d[CATEGORY_d] == TYPE]
        video_name_list=video_type['素材名'].unique()

        new = pd.DataFrame()

        for i in video_name_list:
            #print(i)
            

            if (market_d[market_d['素材名'].str.contains(i)].empty == False):
                #new=new.append(market[market['素材名']==i])
                #new=new.append(market[market['素材名'].str.contains(i)],ignore_index=True)
                new=pd.concat([new,market_d[market_d['素材名'].str.contains(i)]],ignore_index=True)
                new['TYPE']=TYPE
        #print(new.head(),new.describe())
        type_pd_d=pd.concat([type_pd_d,new])


        #new.to_excel(writer,sheet_name=TYPE,index=False)
    return type_pd_d
        


video_new = video[video['制作时间'] >= START_DATE]
video_new = video_new[video_new['制作时间'] <= END_DATE]


video.fillna('无',inplace=True)
video_new.fillna('无',inplace=True)

video=video.drop(video[video['素材名']=='无'].index)
video_new=video_new.drop(video_new[video_new['素材名']=='无'].index)


writer=pd.ExcelWriter(OUTPUT_NAME)


type_pd=get_type(market,video,CATEGORY)
type_pd_new=get_type(market,video_new,CATEGORY)

video_new.to_excel(writer,sheet_name='video_new',index=False)
video.to_excel(writer,sheet_name='video',index=False)


type_pd.to_excel(writer,sheet_name='ALL',index=False)

style_excel(type_pd_new,[u'媒体',u'端',u'TYPE',u'素材名'],"分素材-新",['媒体','端','TYPE','消耗'],[True,True,True,False])    
style_excel(type_pd_new,[u'媒体',u'端',u'TYPE'],"类型-新",['媒体','端','消耗'],[True,True,False]) 

style_excel(market,[u'媒体',u'端',u'素材名'],"市场-素材",['媒体','端','消耗'],[True,True,False]) 
style_excel(market,[u'媒体',u'端'],"市场总",['媒体','消耗'],[True,False]) 

style_excel(type_pd,[u'媒体',u'端',u'TYPE',u'素材名'],"分素材-总",['媒体','端','TYPE','消耗'],[True,True,True,False])    
style_excel(type_pd,[u'媒体',u'端',u'TYPE'],"类型-总",['媒体','端','消耗'],[True,True,False]) 
    



writer.save()


print('done')

C:\Users\liumeishan1\AppData\Local\Temp\ipykernel_8932\2829251595.py:91: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if (market_d[market_d['素材名'].str.contains(i)].empty == False):
C:\Users\liumeishan1\AppData\Local\Temp\ipykernel_8932\2829251595.py:91: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if (market_d[market_d['素材名'].str.contains(i)].empty == False):


分素材-新 done!
类型-新 done!
市场-素材 done!
市场总 done!
分素材-总 done!
类型-总 done!
done


In [45]:
# 朝霞素材统计报表快速筛选

import pandas as pd
import numpy as np
data_list=['头条-安卓','头条-ios','快手-安卓','快手-ios']
df=pd.DataFrame()

def get_list(SUCAI_NAME):

    SUCAI_LOC=SUCAI_NAME+'.xlsx'
    OUTPUT_NAME=SUCAI_NAME+'-output.xlsx'

    data=pd.read_excel(SUCAI_LOC)
    out=pd.DataFrame()

    data=data.drop(data[data['素材名']=='汇总'].index)
    data=data.drop(data[(data['消耗']=='0.00')|(data['消耗']==0)].index)

    MEITI=SUCAI_NAME.split('-')[0]
    DUAN=SUCAI_NAME.split('-')[1]
    data['媒体']=MEITI
    data['端']=DUAN


    out['媒体']=data['媒体']
    out['端']=data['端']
    out['cpsid']=0
    
    out['日期']='221030-221106'


    out['素材名']=data['素材名'].str.replace('.mp4','')
    #out['消耗']=data['消耗'].str.replace(',','').astype(float)
    out['消耗']=data['消耗']
    out['曝光数']=data['展示数']
    out['点击数']=data['点击数']
    out['25%进度播放数']=data['25%进度播放']
    out['50%进度播放数']=data['50%进度播放']
    out['75%进度播放数']=data['75%进度播放']
    out['99%进度播放数']=data['99%进度播放']
    out['有效播放数']=data['有效播放数']

    out['激活数']=data['广告激活数-MP']
    out['注册数']=data['广告注册数-MP']
    out['首日付费人数']=data['首日广告付费账号数-MP']
    # out['次留数']=data['次留数-MP'].str.strip('%').astype(float)/100
    # out['首日付费金额']=data['首日广告付费金额-MP'].astype(str).str.replace(',','').astype(float)*2
    # out['F3付费金额']=data['3日广告付费金额-MP'].astype(str).str.replace(',','').astype(float)*2
    # out['累计付费金额']=data['广告新增付费金额-MP'].astype(str).str.replace(',','').astype(float)*2
    out['次留数']=data['次留数-MP']
    out['首日付费金额']=data['首日广告付费金额-MP'].astype(str).str.replace(',','').astype(float)*2
    out['F3付费金额']=data['3日广告付费金额-MP'].astype(str).str.replace(',','').astype(float)*2
    out['累计付费金额']=data['广告新增付费金额-MP'].astype(str).str.replace(',','').astype(float)*2
    
    writer=pd.ExcelWriter(OUTPUT_NAME)
    out.to_excel(writer,index=False)
    writer.save()
    
    print(SUCAI_NAME,'done')
    return out


new=pd.DataFrame()

for i in data_list:
    new=pd.concat([new,get_list(i)])

writer=pd.ExcelWriter('market.xlsx')
new.to_excel(writer,index=False)
writer.save()
print("completed")




C:\Users\liumeishan1\AppData\Local\Temp\ipykernel_17496\2055421971.py:32: FutureWarning: The default value of regex will change from True to False in a future version.
  out['素材名']=data['素材名'].str.replace('.mp4','')


头条-安卓 done


C:\Users\liumeishan1\AppData\Local\Temp\ipykernel_17496\2055421971.py:32: FutureWarning: The default value of regex will change from True to False in a future version.
  out['素材名']=data['素材名'].str.replace('.mp4','')


头条-ios done


C:\Users\liumeishan1\AppData\Local\Temp\ipykernel_17496\2055421971.py:32: FutureWarning: The default value of regex will change from True to False in a future version.
  out['素材名']=data['素材名'].str.replace('.mp4','')


快手-安卓 done
快手-ios done


C:\Users\liumeishan1\AppData\Local\Temp\ipykernel_17496\2055421971.py:32: FutureWarning: The default value of regex will change from True to False in a future version.
  out['素材名']=data['素材名'].str.replace('.mp4','')


completed
